In [1]:
import sys
import json
import numpy as np
from scipy.optimize import minimize
import time
from itertools import combinations
from config import problem_configs

# Packages for quantum stuff
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit import QuantumCircuit
from qiskit.circuit.library import QAOAAnsatz
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import (
    EstimatorV2 as Estimator,
    QiskitRuntimeService,
    SamplerV2 as Sampler,
)
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime.fake_provider import (
    FakeBrisbane,
    FakeSherbrooke,
    FakeTorino,
)  # For simulation with realistic noise

In [2]:
# //////////    Variables    //////////
reps_p = 20
backend_simulator = AerSimulator()
#backend_simulator = AerSimulator.from_backend(FakeTorino())
instanceOfInterest = 1 #ID for the specific ising model from genereated batch
FILEDIRECTORY = "isingBatches"
classOfInterest = 'MinimumVertexCover'  # Options: 'TSP', 'Knapsack', 'MinimumVertexCover'

In [3]:
#///////////    Configurations    //////////
# Select the configuration based on classOfInterest
try:
    selectedConfig = problem_configs[classOfInterest]
except KeyError:
    raise ValueError(f"Error: '{classOfInterest}' is not a valid problem type. "
                     f"Available options are: {list(problem_configs.keys())}")

isingFileName = f"{FILEDIRECTORY}/batch_Ising_data_{selectedConfig['file_slug']}.json"
exactSolutionsFile = f"{FILEDIRECTORY}/solved_batch_Ising_data_{selectedConfig['file_slug']}.json"
print(isingFileName)

isingBatches/batch_Ising_data_MinimumVertexCover_9q_.json


In [4]:
# //////////    Functions    //////////
def load_ising_and_build_hamiltonian(file_path, instance_id):
    """
    Loads Ising terms and weights from a JSON file.
    Determines the number of qubits from the terms and constructs
    the Hamiltonian as a Qiskit SparsePauliOp.
    """

    with open(file_path, "r") as f:
        all_isings_data = json.load(f)  # Assumes this loads a list of dicts

    selected_ising_data = None
    # Find the desired ising model within list
    for ising_instance in all_isings_data:
        if (
            ising_instance["instance_id"] == instance_id
        ):  # Assumes 'instance_id' exists and is correct
            selected_ising_data = ising_instance
            break

    terms = selected_ising_data["terms"]
    weights = selected_ising_data["weights"]
    problem_type = selected_ising_data.get("problem_type")

    pauli_list = []
    num_qubits = 0

    # Find the max number of qubits by finding the biggest index of ising variables
    all_indices = []
    for term_group in terms:
        for idx in term_group:
            all_indices.append(idx)
    num_qubits = max(all_indices) + 1

    for term_indices, weight in zip(terms, weights):
        paulis_arr = ["I"] * num_qubits
        if len(term_indices) == 1:  # Linear term
            paulis_arr[term_indices[0]] = "Z"
        elif len(term_indices) == 2:  # Quadratic term
            paulis_arr[term_indices[0]] = "Z"
            paulis_arr[term_indices[1]] = "Z"

        pauli_list.append(("".join(paulis_arr)[::-1], weight)) # how from_list works here: https://quantum.cloud.ibm.com/docs/en/api/qiskit/qiskit.quantum_info.SparsePauliOp
    hamiltonian = SparsePauliOp.from_list(pauli_list)
    if problem_type == 'knapsack':
        weight_capacity = selected_ising_data.get("weight_capacity")
        return hamiltonian, num_qubits, problem_type, weight_capacity
    return hamiltonian, num_qubits, problem_type, None

def load_file_into_dict(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    return data

def is_valid_tsp_solution(spin_string):
    try:
        spins = [int(s) for s in spin_string.split(',')]
    except ValueError:
        return False # Handles malformed strings

    # A valid solution must have exactly 9 spins
    if len(spins) != 9:
        return False

    # Constraints on each timestep containing one city
    time_groups = [[0, 1, 2], [3, 4, 5], [6, 7, 8]]
    # Constraints on each city being visited once
    city_groups = [[0, 3, 6], [1, 4, 7], [2, 5, 8]]

    all_groups = city_groups + time_groups

    for group in all_groups:
        # Count how many '-1's are in the current group of spins
        count = sum(1 for index in group if spins[index] == -1)
        
        # If the count is not exactly 1, the constraint is violated
        if count != 1:
            return False

    return True

def calculate_ising_cost(instance_id, spin_string, all_isings_data):
    selected_instance = None
    for instance in all_isings_data:
        if instance["instance_id"] == instance_id:
            selected_instance = instance
            break

    # Parse the input spin string into a list of integers
    spin_values = [int(s) for s in spin_string.split(',')]

    # Iterate through terms and weights to calculate the energy
    total_cost = 0.0
    terms = selected_instance["terms"]
    weights = selected_instance["weights"]

    for term, weight in zip(terms, weights):
        if len(term) == 1:  # Linear term (h_i * s_i)
            i = term[0]
            total_cost += weight * spin_values[i]
        elif len(term) == 2:  # Quadratic term (J_ij * s_i * s_j)
            i, j = term[0], term[1]
            total_cost += weight * spin_values[i] * spin_values[j]

    return total_cost

def convert_basis_to_ising(solution):
    spin_list = ['+1' if bit == '0' else '-1' for bit in solution]
    
    # Join the list elements into a single, comma-separated string
    return ",".join(spin_list)

def build_mixer_hamiltonian(num_qubits, problem_type):
    if problem_type == 'tsp':
        print("Building mixer Hamiltonian for TSP")
        if num_qubits != 9:
            raise ValueError("TSP mixer Hamiltonian only works for exactly 9 qubits.")
        # Each city must be visited once (rows in a 3x3 grid)
        city_constraints = [[0, 1, 2], [3, 4, 5], [6, 7, 8]]
        # Each time step can only have one city (columns in a 3x3 grid)
        time_constraints = [[0, 3, 6], [1, 4, 7], [2, 5, 8]]
        # Combine all constraint groups
        constraints = city_constraints + time_constraints
        pauli_list = []
        for group in constraints:
            # Create pairs of all qubits within the constrained group
            for qubit_pair in combinations(group, 2):
                # Create the XX term
                xx_pauli = ["I"] * num_qubits
                xx_pauli[qubit_pair[0]] = "X"
                xx_pauli[qubit_pair[1]] = "X"
                # Add to the list (in Qiskit's reversed order) with a coefficient of 1.0
                pauli_list.append(("".join(xx_pauli)[::-1], 1.0))

                # Create the YY term
                yy_pauli = ["I"] * num_qubits
                yy_pauli[qubit_pair[0]] = "Y"
                yy_pauli[qubit_pair[1]] = "Y"
                pauli_list.append(("".join(yy_pauli)[::-1], 1.0))
        mixer_hamiltonian = SparsePauliOp.from_list(pauli_list)
        return mixer_hamiltonian
    elif problem_type == 'knapsack':
        print("Building mixer Hamiltonian for Knapsack")
        pauli_list = []
        for i in range(num_qubits):
            # Create an X operator on the i-th qubit
            x_pauli = ["I"] * num_qubits
            x_pauli[i] = "X"
            
            # Add to the list (in Qiskit's reversed order) with a coefficient of 1.0
            pauli_list.append(("".join(x_pauli)[::-1], 1.0))
            
        mixer_hamiltonian = SparsePauliOp.from_list(pauli_list)
        return mixer_hamiltonian
    
    else:
        raise ValueError(f"Unknown problem_type: {problem_type}")
        
def create_inital_state(num_qubits, problem_type, weight_capacity=None):
    """
    Creates an initial state circuit for the given number of qubits and problem type.
    """
    initial_circuit = QuantumCircuit(num_qubits)
    
    if problem_type == 'tsp':
        #starting with simplest obvious scenario, city 0 at time 0, city 1 at time 1, city 2 at time 2
        initial_circuit.x([0, 4, 8])
    elif problem_type == 'knapsack':
        slack_variable_encoding = f'{weight_capacity:0{3}b}'[::-1] #note this only works for knapsack problems with 3 slack variables
        #also note string is reversed to match openQAOA encoding of knapsack problem where leftmost slack variable represents 2^0 remaining capacity, next bit is 2^1, and rightmost slack bit is 2^2
        #this is backwards to normal binary encoding, switcheroo moment
        target_qubits = [index for index, char in enumerate(slack_variable_encoding) if char == '1']
        initial_circuit.x(target_qubits)  # creates the inital state encoding an empty knapsack with slack variables = weight capacity
    else:
        raise ValueError(f"Unknown problem_type: {problem_type}")
    
    return initial_circuit

def cost_func_estimator(
    params, ansatz, estimator, cost_hamiltonian
): 
    global numOptimisations
    transpiledHamil = cost_hamiltonian.apply_layout(ansatz.layout)
    pub = (ansatz, transpiledHamil, params)

    job = estimator.run([pub])
    results = job.result()[0]
    cost = results.data.evs

    cost_float = float(np.real(cost))
    objective_func_vals.append(cost_float)

    numOptimisations = numOptimisations + 1

    return cost_float

In [5]:
costHamil, numQubits, problemType, weightCapacity = load_ising_and_build_hamiltonian(isingFileName, instanceOfInterest)
print(costHamil)
print(f'Problem class is: {problemType}')
if problemType == 'knapsack':
    print(f'Capacity of this knapsack is: {weightCapacity}')


#---- mixer ----
mixerHamil = build_mixer_hamiltonian(numQubits, problemType)
print(mixerHamil)

#--- inital state ---
initialCircuit = create_inital_state(numQubits, problemType, weightCapacity)
print(initialCircuit)

#--- QAOA Ansatz ---
qaoaKwargs = {
    "cost_operator": costHamil,
    "reps": reps_p,
    "initial_state": initialCircuit,
    "mixer_operator": mixerHamil,
}

circuit = QAOAAnsatz(**qaoaKwargs)
circuit.measure_all()
pm = generate_preset_pass_manager(optimization_level=3, backend=backend_simulator)
candidate_circuit = pm.run(circuit)

trainedParamsAndCost = []
num_params = 2 * reps_p
initial_betas = (np.random.rand(reps_p) * np.pi).tolist()
initial_gammas = (np.random.rand(reps_p) * (np.pi)).tolist()
initial_params = initial_betas + initial_gammas #this could be an issue like if you have bad starting parameters
print(initial_params)

objective_func_vals = []
numOptimisations = 0
estimator = Estimator(mode=backend_simulator)

trainResult = minimize(
        cost_func_estimator,
        initial_params,
        args=(candidate_circuit, estimator, costHamil),
        method="COBYLA",  # Using COBYLA for gradient free optimization also fast
        tol=1e-3,
        options={"maxiter": 1000},  # Adjust as needed
    )
print(f'{trainResult}, numLoops: {numOptimisations}') 
trainedParamsAndCost.append([trainResult.x, trainResult.fun])

print(trainedParamsAndCost)


SparsePauliOp(['IIIIIIIZZ', 'IIIIIIZIZ', 'IIIIIIZZI', 'IIIIZZIII', 'IIIZIZIII', 'IIZIIZIII', 'IZIIIZIII', 'ZIIIIZIII', 'IIIZZIIII', 'IIZIZIIII', 'IZIIZIIII', 'ZIIIZIIII', 'IIZZIIIII', 'IZIZIIIII', 'ZIIZIIIII', 'IZZIIIIII', 'ZIZIIIIII', 'ZZIIIIIII', 'IIIIIZIIZ', 'IIIIZIIIZ', 'IIIZIIIIZ', 'IIZIIIIIZ', 'IZIIIIIIZ', 'ZIIIIIIIZ', 'IIIIIZIZI', 'IIIIZIIZI', 'IIIZIIIZI', 'IIZIIIIZI', 'IZIIIIIZI', 'ZIIIIIIZI', 'IIIIIZZII', 'IIIIZIZII', 'IIIZIIZII', 'IIZIIIZII', 'IZIIIIZII', 'ZIIIIIZII', 'IIIIIIIIZ', 'IIIIIIIZI', 'IIIIIIZII', 'IIIIIZIII', 'IIIIZIIII', 'IIIZIIIII', 'IIZIIIIII', 'IZIIIIIII', 'ZIIIIIIII'],
              coeffs=[  10. +0.j,   20. +0.j,   40. +0.j,    5. +0.j,   20. +0.j,   25. +0.j,
   15. +0.j,   25. +0.j,   20. +0.j,   25. +0.j,   15. +0.j,   25. +0.j,
  100. +0.j,   60. +0.j,  100. +0.j,   75. +0.j,  125. +0.j,   75. +0.j,
    5. +0.j,    5. +0.j,   20. +0.j,   25. +0.j,   15. +0.j,   25. +0.j,
   10. +0.j,   10. +0.j,   40. +0.j,   50. +0.j,   30. +0.j,   50. +0.j,
   20. +0.j, 

In [6]:
bestParams = min(trainedParamsAndCost, key=lambda item: item[1])
print(bestParams)
optimized_circuit = candidate_circuit.assign_parameters(bestParams[0])
sampler = Sampler(mode=backend_simulator)
sampler.options.default_shots = 1000

sampleResult = sampler.run([optimized_circuit]).result()
dist = sampleResult[0].data.meas.get_counts()
sortedDist = sorted(dist.items(), key=lambda item: item[1], reverse=True)
print(f'Distribution: {sortedDist}')

[array([0.8889662 , 0.69210012, 4.15573511, 1.72760383, 3.50415174,
       3.3630061 , 2.19409376, 2.30276331, 1.29673304, 1.76890561,
       2.2873683 , 2.51527318, 0.25314954, 3.19609222, 2.56228884,
       1.18973677, 0.13547006, 2.17073391, 3.06379192, 0.90923404,
       1.90874066, 1.09412143, 2.95890228, 0.1250795 , 0.4322944 ,
       0.06042398, 0.59694373, 2.75598456, 2.23146316, 0.2158618 ,
       2.214068  , 0.6332989 , 3.07965626, 2.8879786 , 0.71005356,
       1.00032279, 2.80360672, 1.7466785 , 2.58232878, 2.16965733]), -94.338134765625]
Distribution: [('000011011', 15), ('000100000', 14), ('001011011', 14), ('001101110', 13), ('110010001', 13), ('010010100', 12), ('111100001', 11), ('001011000', 11), ('110001110', 11), ('001011100', 10), ('001001001', 10), ('000110000', 9), ('100010111', 9), ('101111000', 8), ('010111101', 8), ('011101010', 8), ('010110011', 8), ('101010001', 8), ('000000111', 8), ('101011110', 7), ('011011111', 7), ('101101101', 7), ('101001010', 7), ('0

In [7]:
exactSolutions = load_file_into_dict(exactSolutionsFile)
mostProbableSolution = max(dist, key=dist.get)[::-1]  #Reverse the bitstring to match the big-endian convention used by dimod (q_0 leftmost bit)
print(mostProbableSolution)

for item in exactSolutions:
    if item["instance_id"] == int(instanceOfInterest):
        correctSolutionsIsing = item["solutions"]
correctSolutionsBinary = []
for item in correctSolutionsIsing:
    correctSolutionsBinary.append(item.replace('-1', '1').replace('+1', '0').replace(',', '')) #0s and 1s may seem mixed up here, but its because the qubit state |0> corresponds to the Z-eigenvalue of +1, which is the opposite way round to the QUBO>ising mapping
    
print(correctSolutionsBinary)

110110000
['010110010']


In [8]:
#want to compare the cost of solutions and also check whether quantum solution meet constraints
mostProbableIsing = convert_basis_to_ising(mostProbableSolution)
isingModels = load_file_into_dict(isingFileName)

print(f"Highest count solution (in spin variable form): {mostProbableIsing}\nGlobal Optima: {correctSolutionsIsing}")

correctSolutionCost = calculate_ising_cost(instanceOfInterest, correctSolutionsIsing[0], isingModels)
mostProbableCost = calculate_ising_cost(instanceOfInterest, mostProbableIsing, isingModels)
print(f'Cost of highest count solution: {mostProbableCost}\nGlobally minimum cost: {correctSolutionCost}')

if problemType == 'tsp':
    print(f"Conformed to TSP constraints: {is_valid_tsp_solution(mostProbableIsing)}")
print(f"Approximaton Ratio: {mostProbableCost / correctSolutionCost:.2f}") 

Highest count solution (in spin variable form): -1,-1,+1,-1,-1,+1,+1,+1,+1
Global Optima: ['+1,-1,+1,-1,-1,+1,+1,-1,+1']
Cost of highest count solution: -560.5
Globally minimum cost: -605.5
Approximaton Ratio: 0.93
